# WSDM - KKBox's Music Recommendation Challenge

## Description : To predict the chances of a user listening to a song repetitively after the first observable listening event within a time window was triggered

In [1]:
# importing modules

import numpy as np # Mathematical calculation on data
import pandas as pd # Data processing 
import seaborn as sns  # Graph plotting library 
import matplotlib.pyplot as plt # Graph plotting library 
%matplotlib inline # setting plotting inline 



/Users/srimi/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
UsageError: unrecognized arguments: # setting plotting inline

In [3]:
# Checking all the input files in the data folder 
from subprocess import check_output # process library
print(check_output(["ls", "data/"]).decode("utf8"))

members.csv
sample_submission.csv
song_extra_info.csv
songs.csv
test.csv
train.csv



### Exploring the train dataset  

In [12]:
df_train = pd.read_csv("data/train.csv")
df_train.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,target
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1


In [30]:
#plot the target variable first 
plt.figure(figsize=(12,8))
sns.countplot(df_train['target'])

In [10]:
print "Number of unique users is (user_id - msno) : ", df_train['msno'].nunique()
print "Number of unique songs is (song_id): ",df_train['song_id'].nunique()
print "values in source_system_tab : ", list(df_train['source_system_tab'].unique())
print "values in the source_screen_name : ", list(df_train['source_screen_name'].unique())
print "values in the source_type : " ,list(df_train['source_type'].unique())


Number of unique users is (user_id - msno) :  30755
Number of unique songs is (song_id):  359966
values in source_system_tab :  ['explore', 'my library', 'search', 'discover', nan, 'radio', 'listen with', 'notification', 'null', 'settings']
values in the source_screen_name :  ['Explore', 'Local playlist more', nan, 'My library', 'Online playlist more', 'Album more', 'Discover Feature', 'Unknown', 'Discover Chart', 'Radio', 'Artist more', 'Search', 'Others profile more', 'Search Trends', 'Discover Genre', 'My library_Search', 'Search Home', 'Discover New', 'Self profile more', 'Concert', 'Payment']
values in the source_type :  ['online-playlist', 'local-playlist', 'local-library', 'top-hits-for-artist', 'album', nan, 'song-based-playlist', 'radio', 'song', 'listen-with', 'artist', 'topic-article-playlist', 'my-daily-playlist']


In [11]:
# try plotting the source_system_tab value
plt.figure(figsize=(12,12))
sns.countplot(df_train['source_system_tab'])

### Exploring the songs dataset  


In [19]:
df_songs = pd.read_csv("data/songs.csv")
df_songs.head()

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language
0,CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=,247640,465,張信哲 (Jeff Chang),董貞,何啟弘,3.0
1,o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=,197328,444,BLACKPINK,TEDDY| FUTURE BOUNCE| Bekuh BOOM,TEDDY,31.0
2,DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=,231781,465,SUPER JUNIOR,NaN,NaN,31.0
3,dKMBWoZyScdxSkihKG+Vf47nc18N9q4m58+b4e7dSSE=,273554,465,S.H.E,湯小康,徐世珍,3.0
4,W3bqWd3T+VeHFzHAUfARgW9AvVRaF4N5Yzm4Mr6Eo/o=,140329,726,貴族精選,Traditional,Traditional,52.0


In [20]:
# As we have multiple values in genre ids, composer, lyrictist , we can convert them to list
df_songs['genre_ids'] = df_songs.genre_ids.apply(lambda x: str(x).split('|'))
df_songs['composer'] = df_songs.composer.apply(lambda x: str(x).split('|'))
df_songs['lyricist'] = df_songs.lyricist.apply(lambda x: str(x).split('|'))


In [24]:
# Bin the data frame by "song length" with 5 bins...
bins = np.linspace(df_songs.song_length.min(), df_songs.song_length.max(), 10)
df_songs['song_length'] = pd.cut(df_songs.song_length, bins)
plt.figure(figsize=(25,20))
sns.countplot(df_songs.bin_song_length)

# We may outliers in artist name. So we can remove the artist names whose length is too large than 500
df_songs['artist_name'] = df_songs['artist_name'].astype('str')
mask = (df_songs['artist_name'].str.len() < 500)
df_songs = df_songs.loc[mask]


In [26]:
df_songs.head()

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language
0,CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=,"(185, 1352814.444]",[465],張信哲 (Jeff Chang),[董貞],[何啟弘],3.0
1,o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=,"(185, 1352814.444]",[444],BLACKPINK,"[TEDDY, FUTURE BOUNCE, Bekuh BOOM]",[TEDDY],31.0
2,DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=,"(185, 1352814.444]",[465],SUPER JUNIOR,[nan],[nan],31.0
3,dKMBWoZyScdxSkihKG+Vf47nc18N9q4m58+b4e7dSSE=,"(185, 1352814.444]",[465],S.H.E,[湯小康],[徐世珍],3.0
4,W3bqWd3T+VeHFzHAUfARgW9AvVRaF4N5Yzm4Mr6Eo/o=,"(185, 1352814.444]",[726],貴族精選,[Traditional],[Traditional],52.0


In [27]:
# checking the artist songs count by grouping them 
df_songs.groupby(df_songs.artist_name).count()


,song_id,song_length,genre_ids,composer,lyricist,language
artist_name,,,,,,
!Conz,1,1,1,1,1,1
!Creekside,2,2,2,2,2,2
!Dela Dap,40,40,40,40,40,40
!Dris,4,4,4,4,4,4
!Estevan,1,1,1,1,1,1
!Fuckyeah!,1,1,1,1,1,1
!NTELLIGENT Electro Pop,1,1,1,1,1,1
!Ntell!Gent Des!Gn| DJ Shoba,1,1,1,1,1,1
!a!,2,2,2,2,2,2


In [28]:
# check the language column by goruping 
df_songs.groupby(df_songs.language).count()

,song_id,song_length,genre_ids,artist_name,composer,lyricist
language,,,,,,
-1.0,639467,639467,639467,639467,639467,639467
3.0,106295,106295,106295,106295,106295,106295
10.0,15482,15482,15482,15482,15482,15482
17.0,92518,92518,92518,92518,92518,92518
24.0,41744,41744,41744,41744,41744,41744
31.0,39201,39201,39201,39201,39201,39201
38.0,2385,2385,2385,2385,2385,2385
45.0,14435,14435,14435,14435,14435,14435
52.0,1336692,1336690,1336692,1336692,1336692,1336692


## Exploring the members dataset

In [40]:
df_members = pd.read_csv("data/members.csv")
df_members.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34403 entries, 0 to 34402
Data columns (total 7 columns):
msno                      34403 non-null object
city                      34403 non-null int64
bd                        34403 non-null int64
gender                    14501 non-null object
registered_via            34403 non-null int64
registration_init_time    34403 non-null int64
expiration_date           34403 non-null int64
dtypes: int64(5), object(2)
memory usage: 1.8+ MB


In [41]:
#remove outliers in bd columns
#from earlier kernels, I found the column "bd" as "age". Its good to remove outliers in that column
df_members = df_members.query("bd >=5 and bd <80")
df_members.describe()

,city,bd,registered_via,registration_init_time,expiration_date
count,14434.000000,14434.000000,14434.000000,1.443400e+04,1.443400e+04
mean,10.406263,28.855134,6.558612,2.012293e+07,2.017029e+07
std,6.056500,9.017716,2.670512,3.310582e+04,7.832202e+03
min,1.000000,5.000000,3.000000,2.004033e+07,2.005061e+07
25%,5.000000,22.000000,3.000000,2.010122e+07,2.017071e+07
50%,12.000000,27.000000,7.000000,2.013030e+07,2.017092e+07
75%,14.000000,34.000000,9.000000,2.015081e+07,2.017101e+07
max,22.000000,78.000000,13.000000,2.017023e+07,2.020102e+07


In [43]:
# explore tha values of the city
sns.countplot(df_members['city'])

# number of users
print "No of unique members", df_members['msno'].nunique()

# convert the expiration and registration as date type
df_members["registration_init_time"] = pd.to_datetime(df_members["registration_init_time"],format='%Y%m%d', errors='ignore')
df_members["expiration_date"] = pd.to_datetime(df_members["expiration_date"],format='%Y%m%d', errors='ignore')
df_members['days_td'] = df_members['expiration_date'].sub(df_members['registration_init_time'],axis=0)
df_members['days'] = df_members['days_td'].apply( lambda x: x.days )
df_members = df_members.drop(['registration_init_time','expiration_date','days_td'], axis=1)
df_members.head()


No of unique members 14434


,msno,city,bd,gender,registered_via,days
5,zgPOEyUn5a/Fvuzb3m69ajzxjkbblVtObglW89FzLdo=,13,43,female,9,1921
10,GoFYKhcq8Q5Gjz1M5L0azHZOhcY+Za/T7fRqIGgBuYA=,12,28,male,9,4095
13,RoSfblbwJN/izEnFIVw8TgOpm8R/NEpUC84Oz/b32HQ=,15,33,female,3,765
15,vUzJAyFEudsnkWwfcLIKT5mJAV+uVRTW2uajCrghFwI=,13,20,male,3,1113
16,Ev+ouoRTpAjrtxNVeThP6t9xnDG1puQaG28DvJ2db+A=,13,30,male,9,3727


In [44]:
# compare the registered via and days column
df_members_regi = df_members[['registered_via','days']]
df_members_regi.info()

# Reference: easy link to understand grouping in pandas https://chrisalbon.com/python/pandas_apply_operations_to_groups.html

#Checking the average days that user stayed based on their way of registering
print " key - registered via , value - mean for days ", dict(df_members_regi['days'].groupby(df_members_regi['registered_via']).mean())

#Checking the user count based on city 
print " key - city , value - count of number of users",dict(df_members['msno'].groupby(df_members['city']).count())

# check the gender column
print " key - gender , value - count of number of users",dict(df_members['msno'].groupby(df_members['gender']).count())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14434 entries, 5 to 34399
Data columns (total 2 columns):
registered_via    14434 non-null int64
days              14434 non-null int64
dtypes: int64(2)
memory usage: 338.3 KB
 key - registered via , value - mean for days  {9: 2553.5456093062844, 3: 1038.8966131907309, 4: 329.50530255770428, 13: 304.625, 7: 1552.017866811045}
 key - city , value - count of number of users {1: 922, 3: 188, 4: 1568, 5: 2391, 6: 834, 7: 83, 8: 263, 9: 273, 10: 197, 11: 259, 12: 448, 13: 3028, 14: 628, 15: 1375, 16: 31, 17: 138, 18: 244, 19: 10, 20: 26, 21: 191, 22: 1337}
 key - gender , value - count of number of users {'male': 7189, 'female': 6826}


## Exploring the test dataset

In [46]:
df_test = pd.read_csv("data/test.csv")
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2556790 entries, 0 to 2556789
Data columns (total 6 columns):
id                    int64
msno                  object
song_id               object
source_system_tab     object
source_screen_name    object
source_type           object
dtypes: int64(1), object(5)
memory usage: 117.0+ MB


## Merge the songs and members with train dataframe

In [45]:
# Merge the members dataframe into the test dataframe
df_test = pd.merge(left = df_test,right = df_members,how='left',on='msno')
df_test.msno = df_test.msno.astype('category')

# Merge the member dataframe into the train dataframe
df_train = pd.merge(left = df_train,right = df_members,how='left',on='msno')
df_train.msno = df_train.msno.astype('category')



In [49]:
# Merge the Test Dataframe with the SONGS dataframe
df_test = pd.merge(left = df_test,right = df_songs,how = 'left',on='song_id')
df_test.song_id = df_test.song_id.astype('category')

# Merge the Train dataframe with the SONGS dataframe
df_train = pd.merge(left = df_train,right = df_songs,how = 'left',on='song_id')
df_train.song_id = df_train.song_id.astype('category')


In [52]:
# free the memory of df_members, df_songs
del df_members_regi
del df_songs
del df_members

# Train a Light GBM model

In [55]:
import lightgbm as lgb
# SKLEARN
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss

# Create a Cross Validation with 3 splits
kf = KFold(n_splits=3)

# This array will store the predictions made.
predictions = np.zeros(shape=[len(df_test)])

# For each KFold
for train_indices ,validate_indices in kf.split(df_train) : 
    train_data = lgb.Dataset(df_train.drop(['target'],axis=1).loc[train_indices,:],label=df_train.loc[train_indices,'target'])
    val_data = lgb.Dataset(df_train.drop(['target'],axis=1).loc[validate_indices,:],label=df_train.loc[validate_indices,'target'])
    
    # Create the parameters for LGBM
    params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'boosting': 'gbdt',
        'learning_rate': 0.1 ,
        'verbose': 0,
        'num_leaves': 108,
        'bagging_fraction': 0.95,
        'bagging_freq': 1,
        'bagging_seed': 1,
        'feature_fraction': 0.9,
        'feature_fraction_seed': 1,
        'max_bin': 128,
        'max_depth': 10,
        'num_rounds': 200,
        'metric' : 'auc',
        } 
    
    # Train the model
    bst = lgb.train(params, train_data, 100, valid_sets=[val_data])
    
    # Make the predictions storing them on the predictions array
    predictions += bst.predict(df_test.drop(['id'],axis=1))
    
    # Release the model from memory for the next iteration
    del bst

print('Training process finished. Generating Output...')

# We get the ammount of predictions from the prediction list, by dividing the predictions by the number of Kfolds.
predictions = predictions/3

# Read the sample_submission CSV
submission = pd.read_csv(INPUT_DATA_PATH + '/sample_submission.csv')
# Set the target to our predictions
submission.target=predictions
# Save the submission file
submission.to_csv('submission.csv',index=False)

print('Output created.')

OSError: dlopen(/Users/srimi/anaconda/lib/python2.7/site-packages/lightgbm/lib_lightgbm.so, 6): Library not loaded: /usr/local/opt/gcc/lib/gcc/7/libgomp.1.dylib
  Referenced from: /Users/srimi/anaconda/lib/python2.7/site-packages/lightgbm/lib_lightgbm.so
  Reason: image not found